In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# key => 
# 산업안전보건법 + "1"
# 건설기술진흥법 + "2"
# 위험물안전관리법 + '3'
# 전기안전관리법 + "4"
# 화학물질관리법 + '5'

In [2]:
df_industrial = pd.read_excel("산업안전보건법.xlsx")
df_erection = pd.read_excel("건설기술진흥법.xlsx")
df_danger = pd.read_excel("위험물안전관리법.xlsx")
df_electric = pd.read_excel("전기안전관리법.xlsx")
df_chemistry = pd.read_excel("화학물질관리법.xlsx")
df_info = pd.DataFrame()
df_info = df_info.append(df_industrial,ignore_index = True)
df_info = df_info.append(df_erection,ignore_index = True)
df_info = df_info.append(df_danger,ignore_index = True)
df_info = df_info.append(df_electric,ignore_index = True)
df_info = df_info.append(df_chemistry,ignore_index = True)
df_info

,키,조내용
0,10011,제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1,20011,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2,30011,"제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업..."
3,40011,제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4,50011,제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
...,...,...
452,600015,제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453,610015,제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454,620015,제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455,630015,"제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종..."


In [3]:
# key() 메소드에 사용할 id 따로 모음
id=df_info.get("키")
id

# 키를 통해 접근할 수 있도록 키에 번호를 순서대로 부여
num=0
key_dic={}
for i in id:
    key_dic[i]=num
    num+=1

key_dic

{10011: 0,
 20011: 1,
 30011: 2,
 40011: 3,
 50011: 4,
 60011: 5,
 70011: 6,
 80011: 7,
 90011: 8,
 100011: 9,
 110011: 10,
 120011: 11,
 130011: 12,
 140011: 13,
 150011: 14,
 160011: 15,
 170011: 16,
 180011: 17,
 190011: 18,
 200011: 19,
 210011: 20,
 220011: 21,
 230011: 22,
 240011: 23,
 250011: 24,
 260011: 25,
 270011: 26,
 280011: 27,
 290011: 28,
 300011: 29,
 310011: 30,
 320011: 31,
 330011: 32,
 340011: 33,
 350011: 34,
 360011: 35,
 370011: 36,
 380011: 37,
 390011: 38,
 400011: 39,
 410011: 40,
 420011: 41,
 430011: 42,
 440011: 43,
 450011: 44,
 460011: 45,
 470011: 46,
 480011: 47,
 490011: 48,
 500011: 49,
 510011: 50,
 520011: 51,
 530011: 52,
 540011: 53,
 550011: 54,
 560011: 55,
 570011: 56,
 580011: 57,
 590011: 58,
 600011: 59,
 610011: 60,
 620011: 61,
 630011: 62,
 640011: 63,
 650011: 64,
 660011: 65,
 670011: 66,
 680011: 67,
 690011: 68,
 700011: 69,
 710011: 70,
 720011: 71,
 730011: 72,
 740011: 73,
 750011: 74,
 760011: 75,
 770011: 76,
 780011: 77,
 7900

In [4]:
def key(list, y):
    dic = {}
    num=0
    for i in list:
        k=""
        if str(i)[-1]=="1":
            k+="산업안전보건법 제 "+str(i)[0:(len(str(i))-4)]+"조"
        elif str(i)[-1]=="2":
            k+="건설기술진흥법 제 "+str(i)[0:(len(str(i))-4)]+"조"
        elif str(i)[-1]=="3":
            k="위험물안전관리법 제 "+str(i)[0:(len(str(i))-4)]+"조"
        elif str(i)[-1]=="4":
            k="전기안전관리법 제 "+str(i)[0:(len(str(i))-4)]+"조"
        elif str(i)[-1]=="5":
            k="화학물질관리법 제 "+str(i)[0:(len(str(i))-4)]+"조"
        if str(i)[-3]!="0":
            k+="의 "+str(i)[-3]
        dic[k]=y[num]
        num+=1
    return dic

In [5]:
df_info.to_excel('전체법.xlsx',index = False)

In [6]:
# 불용어 리스트
# 불용어, 불필요 단어 제거
stop_words_df = pd.read_excel("stopwords.xlsx")
stop_words_df

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
687,해당
688,포함
689,거나
690,하다


In [7]:
posts = df_info.get("조내용")
posts

0      제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1      제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2      제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업...
3      제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4      제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
                             ...                        
452    제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453    제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454    제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455    제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종...
456    제64조(과태료) 다음 각 호의 어느 하나에 해당하는 자에게는 1천만원 이하의 과태...
Name: 조내용, Length: 457, dtype: object

In [8]:
from konlpy.tag import Okt
okt = Okt()

oo = okt.pos(posts[0],
        norm=True,   # 정규화(normalization)
        stem=True    # 어간추출(stemming)
        )
print(oo)

[('제', 'Noun'), ('1조', 'Number'), ('(', 'Punctuation'), ('목적', 'Noun'), (')', 'Punctuation'), ('이', 'Noun'), ('법', 'Noun'), ('은', 'Josa'), ('산업', 'Noun'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('에', 'Josa'), ('관', 'Noun'), ('한', 'Josa'), ('기준', 'Noun'), ('을', 'Josa'), ('확립', 'Noun'), ('하고', 'Josa'), ('그', 'Noun'), ('책임', 'Noun'), ('의', 'Josa'), ('소재', 'Noun'), ('를', 'Josa'), ('명확하다', 'Adjective'), ('하다', 'Verb'), ('산업', 'Noun'), ('재해', 'Noun'), ('를', 'Josa'), ('예방', 'Noun'), ('하고', 'Josa'), ('쾌적하다', 'Adjective'), ('작업', 'Noun'), ('환경', 'Noun'), ('을', 'Josa'), ('조', 'Modifier'), ('성함', 'Noun'), ('으로써', 'Josa'), ('노무', 'Noun'), ('를', 'Josa'), ('제공', 'Noun'), ('하다', 'Verb'), ('사람', 'Noun'), ('의', 'Josa'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('을', 'Josa'), ('유지', 'Noun'), ('ㆍ', 'Foreign'), ('증진', 'Noun'), ('함', 'Noun'), ('을', 'Josa'), ('목적', 'Noun'), ('으로', 'Josa'), ('하다', 'Verb'), ('.', 'Punctuation'), ('<', 'Punctuation'), ('개정', 'Noun'), ('2020.5', 'Number'), ('.', '

In [9]:
def tokenizer_1(raw_texts, pos=["Noun","Alpha","Verb","Number"], stopword=list(stop_words_df.get("불용어"))):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word not in stopword]
    return(o)

tokenizer_1(posts[23])


['24조',
 '안전',
 '보건',
 '위원회',
 '업주',
 '사업',
 '안전',
 '보건',
 '중요',
 '심의',
 '의결',
 '사업',
 '근로자',
 '위원',
 '사용자',
 '위원',
 '수로',
 '구성',
 '되다',
 '안전',
 '보건',
 '위원회',
 '구성',
 '운영',
 '업주',
 '호의',
 '대해',
 '서다',
 '따르다',
 '안전',
 '보건',
 '위원회',
 '이하',
 '안전',
 '보건',
 '위원회',
 '심의',
 '의결',
 '거치다',
 '안전',
 '보건',
 '위원회',
 '따르다',
 '회의',
 '개최',
 '결과',
 '회의',
 '작성',
 '보존',
 '업주',
 '근로자',
 '따르다',
 '안전',
 '보건',
 '위원회',
 '심의',
 '의결',
 '이행',
 '안전',
 '보건',
 '위원회',
 '따르다',
 '명령',
 '단체',
 '협약',
 '취업규칙',
 '25조',
 '따르다',
 '안전',
 '보건관리',
 '규정',
 '내용',
 '심의',
 '의결',
 '되다',
 '업주',
 '안전',
 '보건',
 '위원회',
 '위원',
 '직무',
 '수행',
 '관련',
 '사유',
 '처우',
 '되다',
 '안전',
 '보건',
 '위원회',
 '구성',
 '사업',
 '종류',
 '사업',
 '근로자',
 '안전',
 '보건',
 '위원회',
 '구성',
 '운영',
 '의결',
 '되다',
 '처리',
 '방법',
 '15조',
 '항제',
 '15조',
 '항제',
 '따르다',
 '중대',
 '재해',
 '기계',
 '기구',
 '설비',
 '도입',
 '안전',
 '보건',
 '관련',
 '조치',
 '사업',
 '근로자',
 '안전',
 '보건',
 '유지',
 '증진']

In [10]:
from konlpy.tag import Mecab
tagger = Mecab()
def tokenizer_2(post):
    nouns = []
  
    for noun in tagger.nouns(post):
        if noun not in list(stop_words_df.get("불용어")):
            nouns.append(noun)
    return nouns

tokenizer_2(posts[23])

['안전',
 '보건',
 '위원회',
 '사업주',
 '사업장',
 '안전',
 '보건',
 '중요',
 '심의',
 '사업장',
 '근로자',
 '위원',
 '사용자',
 '위원',
 '수',
 '구성',
 '안전',
 '보건',
 '위원회',
 '구성',
 '사업주',
 '호',
 '안전',
 '보건',
 '위원회',
 '이하',
 '안전',
 '보건',
 '위원회',
 '심의',
 '안전',
 '보건',
 '위원회',
 '바',
 '회의',
 '개최',
 '결과',
 '회의록',
 '작성',
 '보존',
 '사업주',
 '근로자',
 '안전',
 '보건',
 '위원회',
 '심의',
 '성실',
 '이행',
 '안전',
 '보건',
 '위원회',
 '법',
 '법',
 '명령',
 '단체',
 '협약',
 '취업규칙',
 '안전',
 '보건',
 '관리',
 '규정',
 '내용',
 '심의',
 '사업주',
 '안전',
 '보건',
 '위원회',
 '위원',
 '직무',
 '수행',
 '관련',
 '사유',
 '불리',
 '처우',
 '안전',
 '보건',
 '위원회',
 '구성',
 '사업',
 '종류',
 '사업장',
 '상시',
 '근로자',
 '수',
 '안전',
 '보건',
 '위원회',
 '구성',
 '의결',
 '처리',
 '방법',
 '밖',
 '필요',
 '호',
 '호',
 '호',
 '호',
 '중',
 '중대',
 '재해',
 '유해',
 '위험',
 '기계',
 '도입',
 '안전',
 '보건',
 '관련',
 '조치',
 '밖',
 '사업장',
 '근로자',
 '안전',
 '보건',
 '유지',
 '필요']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize = TfidfVectorizer(
    tokenizer=tokenizer_1, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

X = vectorize.fit_transform(posts)
X.toarray() 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

In [13]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,0.000000,1.289757,1.414214,1.175499,1.248485,1.343371,1.361378,1.361695,1.309491,1.350362,...,1.378859,1.391069,1.398336,1.390360,1.380424,1.414214,1.397602,1.409579,1.414214,1.402701
1,1.289757,0.000000,1.327635,1.336567,1.213869,1.300910,1.341292,1.345794,1.355732,1.229740,...,1.387232,1.388835,1.396432,1.372453,1.384828,1.407918,1.383536,1.388266,1.397695,1.378485
2,1.414214,1.327635,0.000000,1.399259,1.365332,1.386404,1.414214,1.381233,1.414214,1.356870,...,1.379108,1.299542,1.400848,1.378431,1.395492,1.395183,1.395197,1.398395,1.414214,1.395249
3,1.175499,1.336567,1.399259,0.000000,1.300796,1.260531,1.345828,1.291897,1.267851,1.359846,...,1.280975,1.321514,1.395897,1.370300,1.360045,1.402729,1.379076,1.400056,1.392232,1.386225
4,1.248485,1.213869,1.365332,1.300796,0.000000,1.174273,1.365394,1.344439,1.331973,1.319417,...,1.391605,1.398829,1.393355,1.321130,1.336976,1.389226,1.336022,1.354460,1.401320,1.384685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1.414214,1.407918,1.395183,1.402729,1.389226,1.404570,1.408413,1.400976,1.407867,1.378607,...,1.405327,1.377534,1.281977,1.289129,1.346569,0.000000,1.277802,1.309684,1.414214,1.363454
453,1.397602,1.383536,1.395197,1.379076,1.336022,1.369152,1.348739,1.375573,1.381954,1.368932,...,1.401072,1.391729,1.342874,0.977006,1.215813,1.277802,0.000000,1.060126,1.409455,1.074164
454,1.409579,1.388266,1.398395,1.400056,1.354460,1.407796,1.392560,1.388752,1.409989,1.392056,...,1.404550,1.389393,1.363714,1.119819,1.217507,1.309684,1.060126,0.000000,1.373568,1.228051
455,1.414214,1.397695,1.414214,1.392232,1.401320,1.394538,1.414214,1.389627,1.414214,1.414214,...,1.403486,1.394175,1.363075,1.376693,1.357506,1.414214,1.409455,1.373568,0.000000,1.391955


In [14]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
jo_id = 240011

num = key_dic[jo_id]
y_euclidean = euclidean_result_df[num]

dic = key(id, y_euclidean)
    
    
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key = lambda x: x[1])[0:15]

[('산업안전보건법 제 24조', 0.0),
 ('산업안전보건법 제 75조', 0.9339417706976405),
 ('산업안전보건법 제 26조', 1.0332380575761178),
 ('산업안전보건법 제 25조', 1.1056151046058686),
 ('화학물질관리법 제 7조', 1.1500754790486534),
 ('산업안전보건법 제 49조', 1.1758099030796934),
 ('산업안전보건법 제 35조', 1.1796111549562123),
 ('산업안전보건법 제 44조', 1.205628661927156),
 ('건설기술진흥법 제 84조', 1.2072260458479738),
 ('산업안전보건법 제 116조', 1.2217613130534635),
 ('산업안전보건법 제 55조', 1.2279999869394227),
 ('산업안전보건법 제 18조', 1.2283545842192432),
 ('건설기술진흥법 제 5조', 1.2333638966861609),
 ('산업안전보건법 제 22조', 1.2343184588396674),
 ('산업안전보건법 제 13조', 1.242207774097933)]

In [15]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [16]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.168263,0.000000,0.309101,0.220643,0.097677,0.073324,0.072893,0.142616,0.088261,...,0.049374,0.032464,0.022328,0.033449,0.047215,0.000000,0.023354,0.006543,0.000000,0.016215
1,0.168263,1.000000,0.118693,0.106794,0.263261,0.153817,0.100468,0.094419,0.080995,0.243870,...,0.037794,0.035569,0.024989,0.058187,0.041125,0.008884,0.042914,0.036359,0.023224,0.049890
2,0.000000,0.118693,1.000000,0.021037,0.067934,0.038943,0.000000,0.046098,0.000000,0.079452,...,0.049031,0.155595,0.018813,0.049964,0.026300,0.026732,0.026713,0.022246,0.000000,0.026640
3,0.309101,0.106794,0.021037,1.000000,0.153965,0.205530,0.094374,0.165502,0.196276,0.075409,...,0.179552,0.126800,0.025736,0.061139,0.075139,0.016175,0.049075,0.019921,0.030845,0.039190
4,0.220643,0.263261,0.067934,0.153965,1.000000,0.310542,0.067849,0.096242,0.112925,0.129570,...,0.031717,0.021638,0.029281,0.127308,0.106247,0.035025,0.107522,0.082719,0.018151,0.041324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.000000,0.008884,0.026732,0.016175,0.035025,0.013591,0.008186,0.018633,0.008955,0.049721,...,0.012528,0.051200,0.178267,0.169073,0.093376,1.000000,0.183611,0.142364,0.000000,0.070497
453,0.023354,0.042914,0.026713,0.049075,0.107522,0.062712,0.090451,0.053900,0.045102,0.063012,...,0.018498,0.031545,0.098345,0.522730,0.260899,0.183611,1.000000,0.438066,0.006718,0.423086
454,0.006543,0.036359,0.022246,0.019921,0.082719,0.009055,0.030388,0.035684,0.005965,0.031090,...,0.013620,0.034794,0.070142,0.373003,0.258838,0.142364,0.438066,1.000000,0.056655,0.245946
455,0.000000,0.023224,0.000000,0.030845,0.018151,0.027632,0.000000,0.034468,0.000000,0.000000,...,0.015114,0.028139,0.071014,0.052358,0.078588,0.000000,0.006718,0.056655,1.000000,0.031230


In [17]:
# cosine similarity 값으로 
# 24조와 가장 비슷한 조를 찾기위한 과정
y_cosine = cosine_result_df[jo-1]
dic ={}
for i in range(len(y_cosine)):
    dic[i+1] = y_cosine[i]
#24조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:10]

NameError: name 'jo' is not defined

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
countv = CountVectorizer(stop_words = list(stop_words_df.get("불용어")))
sp_mat = countv.fit_transform(posts)
# 희소행렬을 np array로
sp_mat.toarray()

In [ ]:
vectorize = CountVectorizer(
    tokenizer=tokenizer_2, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
)

X = vectorize.fit_transform(posts)
X.toarray() 

In [ ]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

In [ ]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

In [ ]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
num = key_dic[jo_id]
y_euclidean = euclidean_result_df[num]
dic = key(id,y_euclidean)
    
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1])[0:10]

In [ ]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [ ]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

In [ ]:
# cosine similarity 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
num=key_dic[jo_id]
y_cosine = cosine_result_df[num]
dic =dic=key(id, y_cosine)

#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:10]